# Demo-derby

## Data Connections

In [291]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import DivIcon
from shapely.geometry import box
import os
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
import sklearn as skl
import pandasql
from pandasql import sqldf
import psycopg2
import csv
import sqlalchemy    
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from plotly import graph_objects as go
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
import itertools
import scipy.stats as scp
import warnings
import seaborn as sns
sns.set_style("ticks")
#sns.set_context("poster")
from matplotlib import rcParams

rcParams['figure.figsize'] = 32,24

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)

In [ ]:
#----- grabbing our dataframes of demographics and spatial data
df_demos = pd.read_csv('demographics-2016-cleaned.csv', delimiter=",")
with open('neighborhood_feature_set.sql') as f:
    query_feature_set = f.read()
df_demos_cleaned = sqldf(query_feature_set) #this query grabs our feature set specified

tracts = gpd.GeoDataFrame.from_file("vancouver_neighborhoods.geojson")

#----- let's merge our demographic and spatial data into a single frame
df_merged  = tracts.merge(df_demos_cleaned, on='locale', how='left')
gdf_merged = gpd.GeoDataFrame(df_merged)

#----- let's rename the columns for display purposes
gdf_merged=gdf_merged.drop(columns=['percent_suitable_housing', 'percent_no_diploma', 'percent_high_school_diploma'])
gdf_merged.columns = ['locale', 'mapid', 'geometry', '% dwelling in detached home', 
                     'Avg resident age', 'Avg household size', 'Avg gross individual income',
                     'Avg gross household income', 'Avg commute time',
                     '% commuting by car', '% commuting by transit', '% commuting by walk/bike',
                     '% residents homeowners', 'Avg rent', '% residents with post-secondary degree', 
                     '% residents visible minority']

#----- below we rescale all percentage type fields by 100
gdf_merged['% dwelling in detached home'] *=100
gdf_merged['% commuting by car'] *=100
gdf_merged['% commuting by transit'] *=100
gdf_merged['% commuting by walk/bike'] *=100
gdf_merged['% residents homeowners'] *=100
gdf_merged['% residents with post-secondary degree'] *=100
gdf_merged['% residents visible minority'] *=100
gdf_merged=gdf_merged.round(1)

gdf_merged_feature_cols = gdf_merged.drop(columns = ['locale', 'mapid', 'geometry']) # grabbing only graphables

for j in gdf_merged_feature_cols.columns:
#----- let's plot something!
    field_variable = j # this is where the choropleth variable is set

    map_all = folium.Map(location = [49.252, -123.14],
                     max_bounds=True,
                     zoom_start = 12,
                     max_zoom = 12,
                     min_zoom =12,
                     width = '80%',
                     height = '90%',
                     zoom_control=False,dragging=False,
        scrollWheelZoom=False, tiles = 'cartodbpositron')

    folium.Choropleth(
        geo_data=gdf_merged,
        name='choropleth',
        data=gdf_merged,
        columns=['locale', field_variable],
        key_on='feature.properties.locale',
        fill_color='OrRd',
        fill_opacity=0.8,
        line_opacity=0,
        Highlight= True,
        overlay = True,
        smoothfactor = 20
       # legend_name=field_variable
    ).add_to(map_all)

    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}
    on_hover = folium.features.GeoJson(
        gdf_merged,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['locale', field_variable],
            aliases=['', ''],
            style=("background-color: #f5f5f5; color: #fffff;' + \
            'font-family: arial; font-size: 20px; padding: 10px;font-weight:bold;"),
            localize=True
        )
    )
    map_all.add_child(on_hover)
    map_all.keep_in_front(on_hover)

    folium.map.Marker(
        [49.309, -123.27],
        icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=
        '<div style="box-shadow:8px 8px;display:inline-block;padding:8px;'+ \
            'background-color:#f5f5f5;align:left;font-size:16pt"><b>'+field_variable+'</b></div>'
            )
        ).add_to(map_all)
    
    temp_filename = "".join(x for x in field_variable if x.isalnum())
    map_all.save(temp_filename+".html")